In [1]:
# Load autoreload extension
%load_ext autoreload

# Set autoreload behavior
%autoreload 2
import os
import time
import argparse
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from models.ProTACT import ProTACT
from models.Loss import LossFunctions
from configs.configs import Configs
from utils.read_data_pr import read_pos_vocab, read_word_vocab, read_prompts_we, read_essays_prompts, read_prompts_pos
from utils.general_utils import get_scaled_down_scores, pad_hierarchical_text_sequences, get_attribute_masks, load_word_embedding_dict, build_embedd_table
from evaluators.multitask_evaluator_all_attributes import Evaluator as AllAttEvaluator
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /Users/joohwan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/joohwan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# print("main started")
# parser = argparse.ArgumentParser(description="ProTACT model")
# parser.add_argument('--test_prompt_id', type=int,
#                     default=1, help='prompt id of test essay set')
# parser.add_argument('--seed', type=int, default=12, help='set random seed')
# parser.add_argument('--model_name', type=str,
#                     choices=['ProTACT'],
#                     help='name of model')
# parser.add_argument('--num_heads', type=int, default=2,
#                     help='set the number of heads in Multihead Attention')
# parser.add_argument('--features_path', type=str,
#                     default='data/hand_crafted_v3.csv')
test_prompt_id = 1
seed = 1
num_heads = 2
features_path = '../data/hand_crafted_v3.csv'

np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

print(torch.cuda.is_available())
print("Test prompt id is {} of type {}".format(
    test_prompt_id, type(test_prompt_id)))
print("Seed: {}".format(seed))

configs = Configs()

data_path = configs.DATA_PATH
train_path = data_path + str(test_prompt_id) + '/train.pk'
dev_path = data_path + str(test_prompt_id) + '/dev.pk'
test_path = data_path + str(test_prompt_id) + '/test.pk'
pretrained_embedding = configs.PRETRAINED_EMBEDDING
embedding_path = configs.EMBEDDING_PATH
readability_path = configs.READABILITY_PATH
prompt_path = configs.PROMPT_PATH
vocab_size = configs.VOCAB_SIZE
epochs = configs.EPOCHS
batch_size = configs.BATCH_SIZE
print("Numhead : ", num_heads, " | Features : ",
      features_path, " | Pos_emb : ", configs.EMBEDDING_DIM)

read_configs = {
    'train_path': train_path,
    'dev_path': dev_path,
    'test_path': test_path,
    'features_path': features_path,
    'readability_path': readability_path,
    'vocab_size': vocab_size
}

False
Test prompt id is 1 of type <class 'int'>
Seed: 1
Numhead :  2  | Features :  ../data/hand_crafted_v3.csv  | Pos_emb :  50


In [3]:
# read POS for prompts
pos_vocab = read_pos_vocab(read_configs)
prompt_pos_data = read_prompts_pos(
    prompt_path, pos_vocab)  # for prompt POS embedding

# read words for prompts
word_vocab = read_word_vocab(read_configs)
# for prompt word embedding
prompt_data = read_prompts_we(prompt_path, word_vocab)

train_data, dev_data, test_data = read_essays_prompts(
    read_configs, prompt_data, prompt_pos_data, pos_vocab)

if pretrained_embedding:
    embedd_dict, embedd_dim, _ = load_word_embedding_dict(embedding_path)
    embedd_matrix = build_embedd_table(
        word_vocab, embedd_dict, embedd_dim, caseless=True)
    embed_table = embedd_matrix
else:
    embed_table = None

 prompt_pos size: 8
 prompt_words size: 8
 pos_x size: 9513
 readability_x size: 9513
 pos_x size: 1680
 readability_x size: 1680
 pos_x size: 1783
 readability_x size: 1783
Loading GloVe ...
OOV number =189, OOV ratio = 0.047262


In [4]:
max_sentlen = max(train_data['max_sentlen'],
                  dev_data['max_sentlen'], test_data['max_sentlen'])
max_sentnum = max(train_data['max_sentnum'],
                  dev_data['max_sentnum'], test_data['max_sentnum'])
prompt_max_sentlen = prompt_data['max_sentlen']
prompt_max_sentnum = prompt_data['max_sentnum']

print('max sent length: {}'.format(max_sentlen))
print('max sent num: {}'.format(max_sentnum))
print('max prompt sent length: {}'.format(prompt_max_sentlen))
print('max prompt sent num: {}'.format(prompt_max_sentnum))

train_data['y_scaled'] = get_scaled_down_scores(
    train_data['data_y'], train_data['prompt_ids'])
dev_data['y_scaled'] = get_scaled_down_scores(
    dev_data['data_y'], dev_data['prompt_ids'])
test_data['y_scaled'] = get_scaled_down_scores(
    test_data['data_y'], test_data['prompt_ids'])

X_train_pos = pad_hierarchical_text_sequences(
    train_data['pos_x'], max_sentnum, max_sentlen)
X_dev_pos = pad_hierarchical_text_sequences(
    dev_data['pos_x'], max_sentnum, max_sentlen)
X_test_pos = pad_hierarchical_text_sequences(
    test_data['pos_x'], max_sentnum, max_sentlen)

X_train_pos = X_train_pos.reshape(
    (X_train_pos.shape[0], X_train_pos.shape[1] * X_train_pos.shape[2]))
X_dev_pos = X_dev_pos.reshape(
    (X_dev_pos.shape[0], X_dev_pos.shape[1] * X_dev_pos.shape[2]))
X_test_pos = X_test_pos.reshape(
    (X_test_pos.shape[0], X_test_pos.shape[1] * X_test_pos.shape[2]))

X_train_prompt = pad_hierarchical_text_sequences(
    train_data['prompt_words'], max_sentnum, max_sentlen)
X_dev_prompt = pad_hierarchical_text_sequences(
    dev_data['prompt_words'], max_sentnum, max_sentlen)
X_test_prompt = pad_hierarchical_text_sequences(
    test_data['prompt_words'], max_sentnum, max_sentlen)

X_train_prompt = X_train_prompt.reshape(
    (X_train_prompt.shape[0], X_train_prompt.shape[1] * X_train_prompt.shape[2]))
X_dev_prompt = X_dev_prompt.reshape(
    (X_dev_prompt.shape[0], X_dev_prompt.shape[1] * X_dev_prompt.shape[2]))
X_test_prompt = X_test_prompt.reshape(
    (X_test_prompt.shape[0], X_test_prompt.shape[1] * X_test_prompt.shape[2]))

X_train_prompt_pos = pad_hierarchical_text_sequences(
    train_data['prompt_pos'], max_sentnum, max_sentlen)
X_dev_prompt_pos = pad_hierarchical_text_sequences(
    dev_data['prompt_pos'], max_sentnum, max_sentlen)
X_test_prompt_pos = pad_hierarchical_text_sequences(
    test_data['prompt_pos'], max_sentnum, max_sentlen)

X_train_prompt_pos = X_train_prompt_pos.reshape(
    (X_train_prompt_pos.shape[0], X_train_prompt_pos.shape[1] * X_train_prompt_pos.shape[2]))
X_dev_prompt_pos = X_dev_prompt_pos.reshape(
    (X_dev_prompt_pos.shape[0], X_dev_prompt_pos.shape[1] * X_dev_prompt_pos.shape[2]))
X_test_prompt_pos = X_test_prompt_pos.reshape(
    (X_test_prompt_pos.shape[0], X_test_prompt_pos.shape[1] * X_test_prompt_pos.shape[2]))

X_train_linguistic_features = np.array(train_data['features_x'])
X_dev_linguistic_features = np.array(dev_data['features_x'])
X_test_linguistic_features = np.array(test_data['features_x'])

X_train_readability = np.array(train_data['readability_x'])
X_dev_readability = np.array(dev_data['readability_x'])
X_test_readability = np.array(test_data['readability_x'])

Y_train = np.array(train_data['y_scaled'])
Y_dev = np.array(dev_data['y_scaled'])
Y_test = np.array(test_data['y_scaled'])

X_train_attribute_rel = get_attribute_masks(Y_train)
X_dev_attribute_rel = get_attribute_masks(Y_dev)
X_test_attribute_rel = get_attribute_masks(Y_test)

print('================================')
print('X_train_pos: ', X_train_pos.shape)
print('X_train_prompt_words: ', X_train_prompt.shape)
print('X_train_prompt_pos: ', X_train_prompt_pos.shape)
print('X_train_readability: ', X_train_readability.shape)
print('X_train_ling: ', X_train_linguistic_features.shape)
print('X_train_attribute_rel: ', X_train_attribute_rel.shape)
print('Y_train: ', Y_train.shape)

print('================================')
print('X_dev_pos: ', X_dev_pos.shape)
print('X_dev_prompt_words: ', X_dev_prompt.shape)
print('X_dev_prompt_pos: ', X_dev_prompt_pos.shape)
print('X_dev_readability: ', X_dev_readability.shape)
print('X_dev_ling: ', X_dev_linguistic_features.shape)
print('X_dev_attribute_rel: ', X_dev_attribute_rel.shape)
print('Y_dev: ', Y_dev.shape)

print('================================')
print('X_test_pos: ', X_test_pos.shape)
print('X_test_prompt_words: ', X_test_prompt.shape)
print('X_test_prompt_pos: ', X_test_prompt_pos.shape)
print('X_test_readability: ', X_test_readability.shape)
print('X_test_ling: ', X_test_linguistic_features.shape)
print('X_test_attribute_rel: ', X_test_attribute_rel.shape)
print('Y_test: ', Y_test.shape)
print('================================')

max sent length: 50
max sent num: 97
max prompt sent length: 18
max prompt sent num: 8
X_train_pos:  (9513, 4850)
X_train_prompt_words:  (9513, 4850)
X_train_prompt_pos:  (9513, 4850)
X_train_readability:  (9513, 35)
X_train_ling:  (9513, 51)
X_train_attribute_rel:  (9513, 9)
Y_train:  (9513, 9)
X_dev_pos:  (1680, 4850)
X_dev_prompt_words:  (1680, 4850)
X_dev_prompt_pos:  (1680, 4850)
X_dev_readability:  (1680, 35)
X_dev_ling:  (1680, 51)
X_dev_attribute_rel:  (1680, 9)
Y_dev:  (1680, 9)
X_test_pos:  (1783, 4850)
X_test_prompt_words:  (1783, 4850)
X_test_prompt_pos:  (1783, 4850)
X_test_readability:  (1783, 35)
X_test_ling:  (1783, 51)
X_test_attribute_rel:  (1783, 9)
Y_test:  (1783, 9)


In [5]:
# to torch tensor
train_dataset = TensorDataset(
    torch.from_numpy(X_train_pos),
    torch.from_numpy(X_train_prompt),
    torch.from_numpy(X_train_prompt_pos),
    torch.from_numpy(X_train_linguistic_features),
    torch.from_numpy(X_train_readability),
    torch.from_numpy(Y_train)
)
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

dev_dataset = TensorDataset(
    torch.from_numpy(X_dev_pos),
    torch.from_numpy(X_dev_prompt),
    torch.from_numpy(X_dev_prompt_pos),
    torch.from_numpy(X_dev_linguistic_features),
    torch.from_numpy(X_dev_readability),
    torch.from_numpy(Y_dev)
)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

dev_features_list = [
    torch.from_numpy(X_dev_pos),
    torch.from_numpy(X_dev_prompt),
    torch.from_numpy(X_dev_prompt_pos),
    torch.from_numpy(X_dev_linguistic_features),
    torch.from_numpy(X_dev_readability)
]
test_features_list = [
    torch.from_numpy(X_test_pos),
    torch.from_numpy(X_test_prompt),
    torch.from_numpy(X_test_prompt_pos),
    torch.from_numpy(X_test_linguistic_features),
    torch.from_numpy(X_test_readability)
]

In [6]:
# build model
model = ProTACT(
    len(pos_vocab), len(word_vocab), max_sentnum, max_sentlen,
    X_train_readability.shape[1], X_train_linguistic_features.shape[1],
    configs, Y_train.shape[1], num_heads, embed_table
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# loss function and optimizer
criterion = LossFunctions(alpha=0.7)
optimizer = torch.optim.RMSprop(
    model.parameters(), lr=configs.LEARNING_RATE)

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()}")

self.essay_pos_attention_list ModuleList(
  (0-96): 97 x Attention()
)
Layer: essay_pos_embedding.weight | Size: torch.Size([36, 50])
Layer: essay_pos_conv.weight | Size: torch.Size([100, 50, 5])
Layer: essay_pos_conv.bias | Size: torch.Size([100])
Layer: essay_pos_attention_list.0.att_V | Size: torch.Size([100])
Layer: essay_pos_attention_list.0.att_W | Size: torch.Size([100, 100])
Layer: essay_pos_attention_list.1.att_V | Size: torch.Size([100])
Layer: essay_pos_attention_list.1.att_W | Size: torch.Size([100, 100])
Layer: essay_pos_attention_list.2.att_V | Size: torch.Size([100])
Layer: essay_pos_attention_list.2.att_W | Size: torch.Size([100, 100])
Layer: essay_pos_attention_list.3.att_V | Size: torch.Size([100])
Layer: essay_pos_attention_list.3.att_W | Size: torch.Size([100, 100])
Layer: essay_pos_attention_list.4.att_V | Size: torch.Size([100])
Layer: essay_pos_attention_list.4.att_W | Size: torch.Size([100, 100])
Layer: essay_pos_attention_list.5.att_V | Size: torch.Size([100])


In [9]:
evaluator = AllAttEvaluator(
    test_prompt_id, dev_data['prompt_ids'], test_data['prompt_ids'],
    [x.numpy() for x in dev_features_list],
    [x.numpy() for x in test_features_list], Y_dev, Y_test, seed
)

evaluator.evaluate(model, -1, print_info=True)

pos_drop_x:  torch.Size([1680, 4850, 50])
pos_resh_W torch.Size([1680, 97, 50, 50])
pos_zcnn torch.Size([1680, 97, 46, 100])
pos_avg_zcnn torch.Size([1680, 97, 100])
prompt_zcnn torch.Size([1680, 97, 46, 100])
prompt_avg_zcnn torch.Size([1680, 97, 100])
prompt_MA_lstm shape:  torch.Size([1680, 97, 100])
########################
scaled_attention:  tensor([[[[ 0.0233, -0.0094,  0.0141,  ..., -0.1457, -0.0378, -0.0204],
          [-0.0453, -0.0844, -0.0589,  ..., -0.0302,  0.0033,  0.0020]]],


        [[[ 0.0232, -0.0092,  0.0138,  ..., -0.1458, -0.0377, -0.0206],
          [-0.0454, -0.0845, -0.0590,  ..., -0.0301,  0.0033,  0.0022]]],


        [[[ 0.0231, -0.0092,  0.0136,  ..., -0.1459, -0.0377, -0.0208],
          [-0.0454, -0.0843, -0.0591,  ..., -0.0301,  0.0033,  0.0023]]],


        ...,


        [[[ 0.0230, -0.0092,  0.0135,  ..., -0.1458, -0.0378, -0.0208],
          [-0.0454, -0.0843, -0.0591,  ..., -0.0301,  0.0033,  0.0024]]],


        [[[ 0.0231, -0.0092,  0.0134,  ..., 

att_output: torch.Size([1680, 1, 186])
attention_concat: torch.Size([1680, 1, 372])
################################
attention_concat flatten: torch.Size([1680, 372])
tensor([-1.8836e-02, -7.1972e-03,  1.0825e-04, -5.4474e-03,  3.7897e-03,
        -4.0488e-02, -2.3401e-02, -2.5333e-02, -2.6999e-02, -4.2913e-03,
        -1.0909e-02,  2.1828e-03,  2.9504e-02, -1.6529e-03, -6.4342e-03,
         7.7472e-03, -1.8791e-02, -2.3464e-02, -1.6346e-02,  8.6021e-03,
        -6.3106e-03, -2.4100e-03,  3.4562e-02,  1.0701e-02,  1.2906e-02,
         1.7102e-02,  2.3661e-03,  1.3348e-02, -5.6371e-03,  7.5990e-03,
        -1.0034e-02,  1.7085e-02,  5.3278e-02, -1.0580e-02,  1.6893e-03,
         1.9967e-02, -1.9046e-02,  1.7482e-02,  2.6914e-04,  7.4219e-03,
         2.2204e-02, -5.3262e-03,  4.5663e-03, -4.4916e-02, -3.9041e-02,
        -2.1919e-02, -3.2239e-02, -2.3645e-02,  3.1871e-02,  3.4782e-02,
        -3.1820e-02,  5.6933e-03,  1.4822e-02,  2.5069e-02,  1.1422e-02,
        -1.0198e-02,  9.5208e-

pos_drop_x:  torch.Size([1783, 4850, 50])
pos_resh_W torch.Size([1783, 97, 50, 50])
pos_zcnn torch.Size([1783, 97, 46, 100])
pos_avg_zcnn torch.Size([1783, 97, 100])
prompt_zcnn torch.Size([1783, 97, 46, 100])
prompt_avg_zcnn torch.Size([1783, 97, 100])
prompt_MA_lstm shape:  torch.Size([1783, 97, 100])
########################
scaled_attention:  tensor([[[[ 0.0232, -0.0094,  0.0136,  ..., -0.1460, -0.0373, -0.0200],
          [-0.0451, -0.0851, -0.0589,  ..., -0.0301,  0.0033,  0.0020]]],


        [[[ 0.0234, -0.0093,  0.0135,  ..., -0.1459, -0.0374, -0.0198],
          [-0.0450, -0.0852, -0.0589,  ..., -0.0301,  0.0030,  0.0017]]],


        [[[ 0.0231, -0.0095,  0.0139,  ..., -0.1457, -0.0380, -0.0202],
          [-0.0451, -0.0844, -0.0590,  ..., -0.0304,  0.0031,  0.0018]]],


        ...,


        [[[ 0.0235, -0.0094,  0.0142,  ..., -0.1459, -0.0381, -0.0200],
          [-0.0454, -0.0845, -0.0591,  ..., -0.0306,  0.0030,  0.0014]]],


        [[[ 0.0236, -0.0092,  0.0143,  ..., 

att_output: torch.Size([1783, 1, 186])
attention_concat: torch.Size([1783, 1, 372])
################################
attention_concat flatten: torch.Size([1783, 372])
tensor([-1.8803e-02, -7.1988e-03,  9.5597e-05, -5.4604e-03,  3.7986e-03,
        -4.0481e-02, -2.3397e-02, -2.5342e-02, -2.6979e-02, -4.2698e-03,
        -1.0922e-02,  2.1754e-03,  2.9503e-02, -1.6490e-03, -6.4427e-03,
         7.7516e-03, -1.8782e-02, -2.3464e-02, -1.6355e-02,  8.6130e-03,
        -6.2866e-03, -2.4479e-03,  3.4568e-02,  1.0700e-02,  1.2900e-02,
         1.7101e-02,  2.3341e-03,  1.3363e-02, -5.6421e-03,  7.5864e-03,
        -1.0012e-02,  1.7098e-02,  5.3263e-02, -1.0591e-02,  1.7146e-03,
         1.9951e-02, -1.9031e-02,  1.7471e-02,  2.8824e-04,  7.4233e-03,
         2.2190e-02, -5.3386e-03,  4.5387e-03, -4.4893e-02, -3.9042e-02,
        -2.1884e-02, -3.2227e-02, -2.3650e-02,  3.1876e-02,  3.4792e-02,
        -3.1830e-02,  5.7207e-03,  1.4846e-02,  2.5073e-02,  1.1435e-02,
        -1.0193e-02,  9.5291e-

/Users/joohwan/miniforge3/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/joohwan/miniforge3/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


Save best model to checkpoint_best1_1.pth
CURRENT EPOCH: -1
[DEV] AVG QWK: -0.0
[DEV] score QWK: 0.042
[DEV] content QWK: -0.009
[DEV] organization QWK: 0.0
[DEV] word_choice QWK: -0.022
[DEV] sentence_fluency QWK: 0.024
[DEV] conventions QWK: -0.039
[DEV] prompt_adherence QWK: 0.036
[DEV] language QWK: -0.004
[DEV] narrativity QWK: -0.032
------------------------
[TEST] AVG QWK: -0.024
[TEST] score QWK: 0.001
[TEST] content QWK: -0.094
[TEST] organization QWK: 0.063
[TEST] word_choice QWK: -0.125
[TEST] sentence_fluency QWK: 0.0
[TEST] conventions QWK: 0.012
------------------------
[BEST TEST] AVG QWK: -0.024, {epoch}: -1
[BEST TEST] score QWK: 0.001
[BEST TEST] content QWK: -0.094
[BEST TEST] organization QWK: 0.063
[BEST TEST] word_choice QWK: -0.125
[BEST TEST] sentence_fluency QWK: 0.0
[BEST TEST] conventions QWK: 0.012
--------------------------------------------------------------------------------------------------------------------------


In [ ]:

class CustomHistory:
    def __init__(self):
        self.train_loss = []
        self.val_loss = []

    def update(self, train_loss, val_loss):
        self.train_loss.append(train_loss)
        self.val_loss.append(val_loss)
        
        
custom_hist = CustomHistory()

for epoch in range(epochs): # 50
    print(f'Epoch {epoch + 1}/{epochs}')
    start_time = time.time()

    # train
    model.train()
    train_loss = 0.0
    for batch_data in train_loader:
        optimizer.zero_grad()
        batch_data = [x.to(device) for x in batch_data]
        inputs, targets = batch_data[:-1], batch_data[-1]
        outputs = model(*inputs)
        loss = criterion.loss_function(targets.float(),outputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch_data[0].size(0)
    train_loss /= len(train_loader.dataset)

    # validate
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_data in dev_loader:
            batch_data = [x.to(device) for x in batch_data]
            inputs, targets = batch_data[:-1], batch_data[-1]
            outputs = model(*inputs)
            #loss = criterion(outputs, targets)
            loss = criterion.loss_function(targets, outputs)
            val_loss += loss.item() * batch_data[0].size(0)
        val_loss /= len(dev_loader.dataset)

    custom_hist.update(train_loss, val_loss)

    # evaluate
    tt_time = time.time() - start_time
    print(f"Training one epoch in {tt_time:.3f} s")
    evaluator.evaluate(model, epoch + 1)
    print(f"Train Loss: {train_loss:.4f} || Val Loss: {val_loss:.4f}")

evaluator.print_final_info()

'''# show the loss as the graph
fig, loss_graph = plt.subplots()
loss_graph.plot(custom_hist.train_loss,'y',label='train loss')
loss_graph.plot(custom_hist.val_loss,'r',label='val loss')
loss_graph.set_xlabel('epoch')
loss_graph.set_ylabel('loss')
plt.savefig(str('images/protact/test_prompt_'+ str(test_prompt_id) + '_seed_' + str(seed) + '_loss.png'))'''

In [ ]:
Y_train.shape